# Homework 9 - Part 2 Notebook

In this homework notebook, we will create and train our own SkipGram embedding, by using the short synopsis of the Lion King movie explained to kids in the text.text file.

Get familiar with the code and write a small report, with answers to the questions listed at the end of the notebook.

**The report must be submitted in PDF format, before April 8th, 11.59pm!**

Do not forget to write your name and student ID on the report.

### Imports needed

Note, we strongly advise to use a CUDA/GPU machine for this notebook.

Technically, this can be done on CPU only, but it will be very slow!

If you decide to use it on CPU, you might also have to change some of the .cuda() methods used on torch tensors and models in this notebook!

In [1]:
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import functools
import matplotlib.pyplot as plt
CUDA = torch.cuda.is_available()

### Testing for CUDA

We advise running on GPU and setting up CUDA on your machine as it might drastically speed up the running time for this notebook!

In [2]:
# Define device for torch
use_cuda = True
print("CUDA is available:", torch.cuda.is_available())
device = torch.device("cuda" if (use_cuda and torch.cuda.is_available()) else "cpu")

CUDA is available: False


### Step 1. Produce some data based on a given text for training our SkipGram model    

The functions below will be used to produce our dataset for training the SkipGram model.

The dataset text consists of a short description of the story behing the movie The Lion King, explained in simple terms to kids.

In [3]:
def text_to_train(text, context_window):
    """
    This function receives the text as a list of words, in lowercase format.
    It then returns data, a list of all the possible (x,y) pairs with
    - x being the middle word of the sentence of length 2*context_window+1,
    - y being a list of 2k words, containing the k preceding words and the k
    posterior words.
    """
    
    # Get data from list of words in text, using a context window of size k = context_window
    data = []
    for i in range(context_window, len(text) - context_window):
        target = [text[i+e] for e in range(-context_window, context_window+1) if i+e != i]
        input_word = text[i]
        data.append((input_word, target))
        
    return data

In [4]:
def create_text():
    """
    This function loads the string of text from the text.txt file,
    and produces a list of words in string format, as variable text.
    """
    
    # Load corpus from file
    with open("./text.txt", 'r', encoding="utf8",) as f:
        corpus = f.readlines()
    f.close()
    
    # Join corpus into a single string
    text = ""
    for s in corpus:
        l = s.split()
        for s2 in l:
            # Removes all special characters from string
            s2 = ''.join(filter(str.isalnum, s2))
            s2 += ' '
            text += s2.lower()
    text = text.split()
    
    return text

In [5]:
text = create_text()
print(text)

['the', 'lion', 'king', 'is', 'an', 'animated', 'movie', 'made', 'by', 'walt', 'disney', 'in', '1994', 'it', 'was', 'the', 'most', 'successful', 'animated', 'movie', 'of', 'the', '1990s', 'the', 'movie', 'is', 'about', 'a', 'young', 'lion', 'prince', 'who', 'learns', 'about', 'his', 'role', 'as', 'prince', 'and', 'in', 'the', 'circle', 'of', 'life', 'it', 'is', 'dedicated', 'to', 'frank', 'wells', 'who', 'was', 'the', 'president', 'of', 'the', 'walt', 'disney', 'company', 'and', 'died', 'shortly', 'before', 'the', 'movie', 'was', 'released', 'into', 'theaters', 'on', 'june', '15', '1994', 'it', 'was', 'the', 'first', 'fulllength', 'disney', 'movie', 'to', 'feature', 'no', 'human', 'characters', 'since', 'bambi', 'much', 'of', 'the', 'voice', 'acting', 'work', 'was', 'done', 'by', 'wellknown', 'actors', 'including', 'james', 'earl', 'jones', 'jeremy', 'irons', 'matthew', 'broderick', 'whoopi', 'goldberg', 'rowan', 'atkinson', 'jonathan', 'taylor', 'thomas', 'and', 'nathan', 'lane', 'the

In [6]:
def generate_data(text, context_window):
    """
    This function receives the text and context window size.
    It produces four outputs:
    - vocab, a set containing the words found in text.txt,
    without any doublons,
    - word2index, a dictionary to convert words to their integer index,
    - word2index, a dictionary to convert integer index to their respective words,
    - data, containing our (x,y) pairs for training.
    """
    
    # Create vocabulary set V
    vocab = set(text)
    
    # Word to index and index 2 word converters
    word2index = {w:i for i,w in enumerate(vocab)}
    index2word = {i:w for i,w in enumerate(vocab)}
    
    # Generate data
    data = text_to_train(text, context_window)
    
    return vocab, data, word2index, index2word

In [7]:
vocab, data, word2index, index2word = generate_data(text, context_window = 2)

In [8]:
print("The dataset contains", len(vocab), "different words.")

The dataset contains 389 different words.


In [9]:
print("The dadatset contains the following words:")
print(vocab)

The dadatset contains the following words:
{'before', 'much', 'while', 'desert', 'are', 'first', 'done', 'but', 'falls', 'work', '2', 'friend', 'takes', 'can', 'daughter', 'makes', 'liked', 'lies', 'john', 'die', 'rescued', 'puts', 'roars', 'will', 'past', 'his', 'then', 'used', 'ed', 'mother', 'shown', 'forbidden', 'interested', 'which', 'walt', 'show', 'gump', 'picks', 'irons', 'stars', 'feature', 'forrest', 'actors', 'worldwide', 'valley', 'rescues', 'shortly', 'dangerous', 'prepared', 'time', 'scars', 'sarabi', 'opened', 'dead', 'appears', 'killed', 'gather', 'home', 'third', 'atkinson', 'by', 'find', 'there', 'walks', 'became', 'adult', 'simbas', 'company', 'trouble', 'lions', 'nemo', 'died', 'music', 'jonathan', 'back', 'across', 'edge', 'her', 'finds', 'leave', 'fight', 'lot', 'scene', '1990s', 'fights', 'prince', 'than', 'becomes', 'year', 'known', 'uks', 'as', 'animals', 'right', 'towards', 'made', 'like', 'it', 'no', 'three', 'lightning', 'taylor', 'land', 'timon', 'son', 'in

In [10]:
print(word2index)

{'before': 0, 'much': 1, 'while': 2, 'desert': 3, 'are': 4, 'first': 5, 'done': 6, 'but': 7, 'falls': 8, 'work': 9, '2': 10, 'friend': 11, 'takes': 12, 'can': 13, 'daughter': 14, 'makes': 15, 'liked': 16, 'lies': 17, 'john': 18, 'die': 19, 'rescued': 20, 'puts': 21, 'roars': 22, 'will': 23, 'past': 24, 'his': 25, 'then': 26, 'used': 27, 'ed': 28, 'mother': 29, 'shown': 30, 'forbidden': 31, 'interested': 32, 'which': 33, 'walt': 34, 'show': 35, 'gump': 36, 'picks': 37, 'irons': 38, 'stars': 39, 'feature': 40, 'forrest': 41, 'actors': 42, 'worldwide': 43, 'valley': 44, 'rescues': 45, 'shortly': 46, 'dangerous': 47, 'prepared': 48, 'time': 49, 'scars': 50, 'sarabi': 51, 'opened': 52, 'dead': 53, 'appears': 54, 'killed': 55, 'gather': 56, 'home': 57, 'third': 58, 'atkinson': 59, 'by': 60, 'find': 61, 'there': 62, 'walks': 63, 'became': 64, 'adult': 65, 'simbas': 66, 'company': 67, 'trouble': 68, 'lions': 69, 'nemo': 70, 'died': 71, 'music': 72, 'jonathan': 73, 'back': 74, 'across': 75, 'ed

In [11]:
print(index2word)

{0: 'before', 1: 'much', 2: 'while', 3: 'desert', 4: 'are', 5: 'first', 6: 'done', 7: 'but', 8: 'falls', 9: 'work', 10: '2', 11: 'friend', 12: 'takes', 13: 'can', 14: 'daughter', 15: 'makes', 16: 'liked', 17: 'lies', 18: 'john', 19: 'die', 20: 'rescued', 21: 'puts', 22: 'roars', 23: 'will', 24: 'past', 25: 'his', 26: 'then', 27: 'used', 28: 'ed', 29: 'mother', 30: 'shown', 31: 'forbidden', 32: 'interested', 33: 'which', 34: 'walt', 35: 'show', 36: 'gump', 37: 'picks', 38: 'irons', 39: 'stars', 40: 'feature', 41: 'forrest', 42: 'actors', 43: 'worldwide', 44: 'valley', 45: 'rescues', 46: 'shortly', 47: 'dangerous', 48: 'prepared', 49: 'time', 50: 'scars', 51: 'sarabi', 52: 'opened', 53: 'dead', 54: 'appears', 55: 'killed', 56: 'gather', 57: 'home', 58: 'third', 59: 'atkinson', 60: 'by', 61: 'find', 62: 'there', 63: 'walks', 64: 'became', 65: 'adult', 66: 'simbas', 67: 'company', 68: 'trouble', 69: 'lions', 70: 'nemo', 71: 'died', 72: 'music', 73: 'jonathan', 74: 'back', 75: 'across', 76:

In [12]:
print(data)

[('king', ['the', 'lion', 'is', 'an']), ('is', ['lion', 'king', 'an', 'animated']), ('an', ['king', 'is', 'animated', 'movie']), ('animated', ['is', 'an', 'movie', 'made']), ('movie', ['an', 'animated', 'made', 'by']), ('made', ['animated', 'movie', 'by', 'walt']), ('by', ['movie', 'made', 'walt', 'disney']), ('walt', ['made', 'by', 'disney', 'in']), ('disney', ['by', 'walt', 'in', '1994']), ('in', ['walt', 'disney', '1994', 'it']), ('1994', ['disney', 'in', 'it', 'was']), ('it', ['in', '1994', 'was', 'the']), ('was', ['1994', 'it', 'the', 'most']), ('the', ['it', 'was', 'most', 'successful']), ('most', ['was', 'the', 'successful', 'animated']), ('successful', ['the', 'most', 'animated', 'movie']), ('animated', ['most', 'successful', 'movie', 'of']), ('movie', ['successful', 'animated', 'of', 'the']), ('of', ['animated', 'movie', 'the', '1990s']), ('the', ['movie', 'of', '1990s', 'the']), ('1990s', ['of', 'the', 'the', 'movie']), ('the', ['the', '1990s', 'movie', 'is']), ('movie', ['19

In [ ]:
def words_to_tensor(words: list, word2index: dict, dtype = torch.FloatTensor):
    """
    This function converts a word or a list of words into a torch tensor,
    with appropriate format.
    It reuses the word2index dictionary.
    """
    
    tensor = dtype([word2index[word] for word in words])
    tensor = tensor.to(device)
    
    return Variable(tensor)

### Step 2. Create a SkipGram model and train

Task #1: Write your own model for the SkipGram model below.

In [ ]:
class SkipGram(nn.Module):
    """
    Your skipgram model here!
    """
    
    def __init__(self, context_size, embedding_dim, vocab_size):
        pass

    def forward(self, inputs):
        pass

In [ ]:
# Create model and pass to CUDA if available.
model = SkipGram(context_size = 2, embedding_dim = 20, vocab_size = len(vocab))
model = model.to(device)
model.train()

In [ ]:
# Define training parameters
learning_rate = 0.001
epochs = 50
torch.manual_seed(28)
loss_function = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr = learning_rate)

Task #2: Write your own training function for the SkipGram model in the cell below. It should return a list of losses and accuracies for display later on, along with your trained model. You may also write a helper function for computing the accuracy of your model during training.

In [ ]:
def get_prediction(target, model, word2index, index2word):
    """
    This is a helper function to get predictions from our model.
    """
    
    return None

In [ ]:
def train(data, word2index, model, epochs, loss_func, optimizer):
    """
    This is a trainer function to train our SkipGram model.
    """
    losses = []
    accuracies = []
    pass
    return losses, accuracies, model

losses, accuracies, model = train(data, word2index, model, epochs, loss_func, optimizer)

### 3. Visualization

In [ ]:
# Display losses over time
plt.figure()
plt.plot(losses)
plt.show()

In [ ]:
# Display accuracies over time
plt.figure()
plt.plot(accuracies)
plt.show()

### 4. Extract embedding and play with it? (Optional)

### Questions and expected answers for the report

The questions listed below are related to Part 2.

P2-QA. Copy and paste your SkipGram class code (Task #1 in this notebook).

P2-QB. Copy and paste your train function (Task #2 in the notebook), along with any helper functions you might have used (e.g. a function to compute the accuracy of your model after each iteration).
Please also copy and paste the function call with the parameters you used for the train() function.

P2-QC. Why is the SkipGram model much more difficult to train than the CBoW?
Is it problematic if it does not reach a 100% accuracy on the task it is being trained on?

P2-QD. If we were to evaluate this model by using intrinsic methods, what could be a possible approach to do so?

P2-QE. (Optional) Please submit any additional code you might that will demonstrate the performance/problems of the word embedding you have trained!